In [1]:
import os
import sys
import pandas as pd
sys.path.append("../")
from dotenv import load_dotenv
from big_query import UniswapV3Data
load_dotenv()

True

In [2]:
univ3_data = UniswapV3Data(path='../data/', project_id=os.getenv("PROJECT_ID"), key_path=os.getenv("KEY_PATH"))
ETH_USDC_ADDRESS = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"
swap_data = univ3_data.get_swap_data(address=ETH_USDC_ADDRESS, download_latest=False)
swap_data.head()

,block_timestamp,block_number,transaction_hash,log_index,contract_address,sender,recipient,amount0,amount1,sqrtPriceX96,liquidity,tick
0,2021-05-05 01:56:23+00:00,12371376,0xce7c3c307d820785caa12938012372fc9366a614a6aa...,26,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xe592427a0aece92de3edee1f18e0157c05861564,0x5eefc9306f11a824762ccdaedac41049efc7fcc8,-329608,100000000000000,1377932816571815120446551350158799,4303369674465501,195285
1,2021-05-05 08:23:26+00:00,12373132,0x9a1c51b0bffbf840948f3b6e3f3e495ba1cd3fa64854...,192,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,-164694492,50000000000000000,1378850591292581266780357299649652,4303369674465501,195298
2,2021-05-05 09:50:51+00:00,12373520,0xc58715c62a5bf70a6ca09f0e51546d6cad76c8d4fff0...,8,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xe592427a0aece92de3edee1f18e0157c05861564,0xfb36693ac2dbe8dfc9f89dd0de6015c6ea66b0bf,-329169,100000000000000,1378852426842022799073024911548633,4303369674465501,195298
3,2021-05-05 11:59:57+00:00,12374077,0x288c21b8b4fbf449b1d086a06e43b124ac2bc088c3f5...,86,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xe592427a0aece92de3edee1f18e0157c05861564,0xfb36693ac2dbe8dfc9f89dd0de6015c6ea66b0bf,2,-329169,1378852426842016741051966412054516,4304946248093346,195298
4,2021-05-05 12:56:56+00:00,12374320,0x67502d8ba373287f6d301f6baa77c5a5f4c80d0753c3...,257,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,0xe592427a0aece92de3edee1f18e0157c05861564,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,1559137299,-467880854065813753,1370241555019945317645788135487819,4304946248093346,195173


In [3]:
# start_block, end_block = swap_data.block_number.iloc[0], swap_data.block_number.iloc[-1] 
# gas_price = univ3_data.get_gas_data(start_block, end_block)
# gas_price.head()

```Based on historic vol you can actually know what price what's the propability that our option expires ITM or OTM```

In [4]:
swap_data.block_timestamp =  pd.to_datetime(swap_data['block_timestamp'], origin='unix')
swap_data.set_index('block_timestamp', inplace=True)
DATE_BEGIN                 = pd.to_datetime('2021-07-15 00:00PM',utc=True)
DATE_END                   = pd.to_datetime('2021-08-15 00:00PM',utc=True)
swap_data = swap_data[DATE_BEGIN:DATE_END]
DECIMALS_0 = 6
DECIMALS_1 = 18
swap_data['amount0']  = swap_data.amount0.astype('float')/(10**DECIMALS_0)
swap_data['amount1']  = swap_data.amount1.astype('float')/(10**DECIMALS_1)
swap_data['liquidity']  = swap_data.liquidity.astype('float')
swap_data['traded_in'] = swap_data.apply(lambda x: -x['amount0'] if (x['amount0'] < 0) else -x['amount1'],axis=1)
swap_data['traded_out'] = swap_data.apply(lambda x:  x['amount0'] if (x['amount0'] > 0) else  x['amount1'],axis=1)
swap_data['token_in']        = swap_data.apply(lambda x: 'token0' if (x['amount0'] < 0) else 'token1',axis=1)


price_data = pd.Series((1.0001**swap_data.tick.values)/10**(DECIMALS_1-DECIMALS_0),index=swap_data.index)

In [5]:
from rolling_strategy import RollingStrategy
import strategy_state 

Strategy = RollingStrategy(duration_param='14 day', percentage_param=0.3, buffer_param='7 day')
# Initial Position Details
INITIAL_TOKEN_0            = 100000
INITIAL_TOKEN_1            = INITIAL_TOKEN_0*price_data[0]
INITIAL_POSITION_VALUE     = 2*INITIAL_TOKEN_0
FEE_TIER                   = 0.003

output = strategy_state.simulate_strategy(price_data,swap_data,Strategy,INITIAL_TOKEN_0,INITIAL_TOKEN_1,fee_tier=FEE_TIER,decimals_0=DECIMALS_0,decimals_1=DECIMALS_1)

Initializing strategy...
Setting liquidity ranges: 9246867203082602
Setting liquidity ranges: 18163841622879860
Setting liquidity ranges: 8849904261690618
Setting liquidity ranges: 15775213516290122
Setting liquidity ranges: 7453346626106560


In [6]:
sim_data = strategy_state.generate_simulation_series(output,Strategy)
strat_result = strategy_state.analyze_strategy(sim_data,INITIAL_POSITION_VALUE)

In [7]:
strat_result

{'days_strategy': 30,
 'gross_fee_apr': 0.18064486122724535,
 'gross_fee_return': 0.014847522840595508,
 'net_apr': -2.9332067620608018,
 'net_return': -0.24108548729266865,
 'rebalances': 4,
 'max_drawdown': 0.6725567234277957,
 'volatility': 9.521073079981054,
 'sharpe_ratio': -0.3080752282248671,
 'impermanent_loss': 2.0356580508293254,
 'mean_first_position': 0.4822540403091458,
 'median_first_position': 0.33882113494580574,
 'final_value': 151782.90254146626}

In [8]:
sim_data

,time,price,reset_point,reset_reason,first_position_lower,first_position_upper,second_position_lower,second_position_upper,token_0_fees,token_1_fees,...,value_position,value_allocated,value_left_over,first_position_value,second_position_value,value_position_usd,cum_fees_usd,token_0_hold_usd,token_1_hold_usd,value_hold_usd
time,,,,,,,,,,,,,,,,,,,,,
2021-07-15 12:01:11+00:00,2021-07-15 12:01:11+00:00,0.000523,False,,0.000523,0.000680,0.000523,0.000680,0.000000,0.000000,...,50000.000000,50000.000000,0.0,50000.000000,0.0,50000.000000,0.000000,50000.0,0.0,50000.0
2021-07-15 12:01:30+00:00,2021-07-15 12:01:30+00:00,0.000523,False,,0.000523,0.000680,0.000523,0.000680,3.050945,0.000000,...,50003.050945,50000.000000,0.0,50000.000000,0.0,50003.050945,3.050945,50000.0,0.0,50000.0
2021-07-15 12:01:30+00:00,2021-07-15 12:01:30+00:00,0.000525,False,,0.000523,0.000680,0.000523,0.000680,3.050945,0.000000,...,50004.165629,49998.063740,0.0,49998.063740,0.0,50004.165629,6.101890,50000.0,0.0,50000.0
2021-07-15 12:03:14+00:00,2021-07-15 12:03:14+00:00,0.000524,False,,0.000523,0.000680,0.000523,0.000680,3.050945,0.000574,...,50010.248191,50000.000000,0.0,50000.000000,0.0,50010.248191,10.248191,50000.0,0.0,50000.0
2021-07-15 12:03:24+00:00,2021-07-15 12:03:24+00:00,0.000524,False,,0.000523,0.000680,0.000523,0.000680,0.000000,0.000809,...,50011.793250,50000.000000,0.0,50000.000000,0.0,50011.793250,11.792373,50000.0,0.0,50000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-15 11:31:34+00:00,2021-08-15 11:31:34+00:00,0.000317,False,,0.000322,0.000419,0.000380,0.000495,0.000000,0.000000,...,151782.630017,151387.203684,0.0,51387.203684,100000.0,151782.630017,2969.504568,50000.0,0.0,50000.0
2021-08-15 11:32:11+00:00,2021-08-15 11:32:11+00:00,0.000317,False,,0.000322,0.000419,0.000380,0.000495,0.000000,0.000000,...,151782.805168,151387.203684,0.0,51387.203684,100000.0,151782.805168,2969.504568,50000.0,0.0,50000.0
2021-08-15 11:32:58+00:00,2021-08-15 11:32:58+00:00,0.000317,False,,0.000322,0.000419,0.000380,0.000495,0.000000,0.000000,...,151782.902541,151387.203684,0.0,51387.203684,100000.0,151782.902541,2969.504568,50000.0,0.0,50000.0
